In [16]:
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time
from tqdm import tqdm_notebook

from scipy.stats import spearmanr, kendalltau
from scipy import linalg

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture as GMM

np.set_printoptions(precision=4, suppress=True)

## Спортивное "Что? Где? Когда?": данные

In [90]:
import json, pickle

tournaments = pickle.load(open('tournaments.pkl', 'rb'))
results = pickle.load(open('chgk/results.pkl', 'rb'))
players = pickle.load(open('players.pkl', 'rb'))

#### 1. Считывание и анализ данных

План действий: 
- выбор данных за 2019 и 2020 гг.
- построение датасета для обучения baseline-модели
  - колонки: ['Start_rating', 'Answered', 'Failed', 'Final_rating'], где \
    первая и последняя колонки отвечают за рейтинг из первого и последнего соревнований 2019г., \
    а вторая и третья за количество отвеченных и неотвеченных вопросов \
- построение датасета для обучения ЕМ-модели 
  - колонки: ['round', 'team', 'mask'], где \
  round - номер турнира \
  team - id участников команды \
  mask - список ответов

In [3]:
tournaments2019 = [v['id'] for k, v in tournaments.items() if '2019' in v['dateStart']]
tournaments2020 = [v['id'] for k, v in tournaments.items() if '2020' in v['dateStart']]

In [4]:
tournaments[4772]

{'id': 4772,
 'name': 'Синхрон северных стран. Зимний выпуск',
 'dateStart': '2019-01-05T19:00:00+03:00',
 'dateEnd': '2019-01-09T19:00:00+03:00',
 'type': {'id': 3, 'name': 'Синхрон'},
 'season': '/seasons/52',
 'orgcommittee': [{'id': 28379,
   'name': 'Константин',
   'patronymic': 'Владимирович',
   'surname': 'Сахаров'}],
 'synchData': {'dateRequestsAllowedTo': '2019-01-09T23:59:59+03:00',
  'resultFixesTo': '2019-01-19T23:59:59+03:00',
  'resultsRecapsTo': '2019-01-11T23:59:59+03:00',
  'allowAppealCancel': True,
  'allowNarratorErrorAppeal': False,
  'dateArchivedAt': '2019-01-26T23:59:59+03:00',
  'dateDownloadQuestionsFrom': '2019-01-04T00:00:00+03:00',
  'dateDownloadQuestionsTo': '2019-01-09T19:00:00+03:00',
  'hideQuestionsTo': '2019-01-09T23:59:59+03:00',
  'hideResultsTo': '2019-01-09T23:59:59+03:00',
  'allVerdictsDone': None,
  'instantControversial': True},
 'questionQty': {'1': 12, '2': 12, '3': 12}}

In [19]:
results[5465][0]['mask'], [(p['player']['id'], p['rating']) for p in results[5465][0]['teamMembers']]

('010111101111010110001000111101011010111011000111111111110011110100001010101111111111011110',
 [(28751, 14230),
  (30152, 14230),
  (30270, 14230),
  (27822, 14168),
  (27403, 14021),
  (4270, 13945)])

***Преобразование в датасет для обучения baseline-модели***

In [13]:
from itertools import chain

In [22]:
results2019 = OrderedDict()
results2019 = {k: v for k, v in results.items() if k in tournaments2019}
results2020 = OrderedDict()
results2020 = {k: v for k, v in results.items() if k in tournaments2020}

In [27]:
with open('results2019.pkl', 'wb') as f:
    pickle.dump(results2019, f)
    
with open('results2020.pkl', 'wb') as f:
    pickle.dump(results2020, f)

***Построение датасета для обучения ЕМ-модели***

In [ ]:
results2019 = pickle.load(open('results2019.pkl', 'rb'))
results2020 = pickle.load(open('results2020.pkl', 'rb'))

In [12]:
players_results2019 = pd.DataFrame(index=range(1, 250000), columns=['StartRating', 
                                                                    'Answered', 
                                                                    'Failed', 
                                                                    'FinalRating'])
players_results2019['Answered'] = 0
players_results2019['Failed'] = 0

players_results2020 = pd.DataFrame(index=range(1, 250000), columns=['StartRating', 
                                                                    'Answered', 
                                                                    'Failed', 
                                                                    'FinalRating'])
players_results2020['Answered'] = 0
players_results2020['Failed'] = 0

In [15]:
for key, result in tqdm_notebook(results2019.items()):
    for team in result:
        for p in team['teamMembers']:
            if 'questionsTotal' in team.keys() and 'mask' in team.keys():
                if team['mask'] is not None:
                    if pd.isnull(players_results2019['StartRating'].iloc[p['player']['id'] - 1]):
                        players_results2019['StartRating'].iloc[p['player']['id'] - 1] = p['rating']

                    players_results2019['FinalRating'].iloc[p['player']['id'] - 1] = p['rating']
                    
                    players_results2019['Answered'].iloc[p['player']['id'] - 1] += team['questionsTotal']
                    players_results2019['Failed'].iloc[p['player']['id'] - 1] += team['questionsTotal'] - len(team['mask'])

  0%|          | 0/687 [00:00<?, ?it/s]

In [16]:
for key, result in tqdm_notebook(results2020.items()):
    for team in result:
        for p in team['teamMembers']:
            if 'questionsTotal' in team.keys() and 'mask' in team.keys():
                if team['mask'] is not None:
                    if pd.isnull(players_results2020['StartRating'].iloc[p['player']['id'] - 1]):
                        players_results2020['StartRating'].iloc[p['player']['id'] - 1] = p['rating']

                    players_results2020['FinalRating'].iloc[p['player']['id'] - 1] = p['rating']
                    
                    players_results2020['Answered'].iloc[p['player']['id'] - 1] += team['questionsTotal']
                    players_results2020['Failed'].iloc[p['player']['id'] - 1] += team['questionsTotal'] - len(team['mask'])

  0%|          | 0/418 [00:00<?, ?it/s]

In [17]:
players_results2019.dropna(how='any', inplace=True)
players_results2020.dropna(how='any', inplace=True)

In [18]:
with open('players_results2019.pkl', 'wb') as f:
    pickle.dump(players_results2019, f)
    
with open('players_results2020.pkl', 'wb') as f:
    pickle.dump(players_results2020, f)

In [2]:
with open('players_results2019.pkl', 'rb') as f:
    players_results2019 = pickle.load(f)
    
with open('players_results2020.pkl', 'rb') as f:
    players_results2020 = pickle.load(f)

In [11]:
players_results2019.head()

,StartRating,Answered,Failed,FinalRating
15,5077,202,-360,7841
16,4247,129,-105,3301
23,791,18,-18,791
31,4340,464,-586,5752
35,5742,480,-447,6717


***3. Обучение baseline-модели***
- обучение при помощи линейной регрессии
- признаковые переменные: начальный рейтинг, количество отвеченных, количество неотвеченных вопросов
- целевая переменная - конечный рейтинг игрока в 2019г.

In [21]:
lin_reg = LinearRegression()
lin_reg.fit(players_results2019[['StartRating', 'Answered', 'Failed']], players_results2019['FinalRating'])
predicted = lin_reg.predict(players_results2020[['StartRating', 'Answered', 'Failed']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

***Результаты работы baseline-модели***

In [29]:
# Корреляция Спирмена
spearmanr(players_results2020['FinalRating'], predicted)

SpearmanrResult(correlation=0.7621661009238474, pvalue=0.0)

In [30]:
# Корреляция Кендалла
kendalltau(players_results2020['FinalRating'], predicted)

KendalltauResult(correlation=0.7145138386446295, pvalue=0.0)

***Построение датасетов для обучения ЕМ-модели***
Вид датасета: 
 - колонки: ['round', 'team', 'mask'], где \
  round - номер турнира \
  team - id участников команды \
  mask - список ответов

In [13]:
keys2019 = [k for k, v in results2019.items()]

In [17]:
teams_masks = []

for key, result in tqdm_notebook(results2019.items()):
    for team in result:
        
        members = []
        if 'mask' in team.keys():
            if team['mask'] is not None:
                mask = team['mask']
                
                for p in team['teamMembers']:
                    members.append(p['player']['id'])
                    
                teams_masks.append((key, members, mask))

In [18]:
rounds = [tourn[0] for tourn in teams_masks] 
teams = [team[1] for team in teams_masks]
masks = [mask[2] for mask in teams_masks]

In [19]:
masks2019 = pd.DataFrame(index=range(len(teams)), columns=['round', 'team', 'mask'])
masks2019['round'] = rounds
masks2019['team'] = teams
masks2019['mask'] = masks

In [25]:
# Датасет для 2019г.
masks2019.head()

,round,team,mask
0,4772,"[6212, 18332, 18036, 22799, 15456, 26089]",111111111011111110111111111100010010
1,4772,"[1585, 40840, 1584, 10998, 16206]",111111111011110100101111011001011010
2,4772,"[23513, 18168, 21060, 35850, 31332, 10187]",111111111011110101101111001011110000
3,4772,"[36742, 28939, 54289, 15381, 27375]",101111101111111110001101011001111010
4,4772,"[28689, 17720, 30597, 12400, 26988, 69476]",111111101011111101000111001001111110


In [20]:
keys2020 = [k for k, v in results2020.items()]

In [21]:
teams_masks2020 = []

for key, result in tqdm_notebook(results2020.items()):
    for team in result:
        
        members = []
        if 'mask' in team.keys():
            if team['mask'] is not None:
                mask = team['mask']
                
                for p in team['teamMembers']:
                    members.append(p['player']['id'])
                    
                teams_masks2020.append((key, members, mask))

In [22]:
rounds_2020 = [tourn[0] for tourn in teams_masks2020] 
teams_2020 = [team[1] for team in teams_masks2020]
masks_2020 = [mask[2] for mask in teams_masks2020]

In [23]:
masks2020 = pd.DataFrame(index=range(len(teams_2020)), columns=['round', 'team', 'mask'])
masks2020['round'] = rounds_2020
masks2020['team'] = teams_2020
masks2020['mask'] = masks_2020

In [31]:
# Датасет для 2020г.
masks2020.head()

,round,team,mask
0,4957,"[30152, 30270, 27822, 28751, 27403, 4270]",111111011111110110X11101111000101010001
1,4957,"[34936, 40877, 25177, 113703, 33792, 107161]",101111011111110100X00111111011100010100
2,4957,"[33620, 21346, 13857, 46339, 37836, 19632]",101110101011010000X11111111110100000111
3,4957,"[32901, 28689, 19541, 13689, 9801, 18194]",111101001110010110X11011111100000000111
4,4957,"[6482, 34846, 36120, 32458, 25882, 30475]",001110000011110111X01011111000100010111


#### 4. Построение схемы ЕМ-модели
В схеме для обучения на Е-шаге использовалась логистическая регрессия.
На М-шаге веса логистической регрессии умножались на текущие рейтинги и складывались с ними. 

Датасет для обучения логистической регрессии: 
Колонки: id игроков
Индексы: команды
Целевая переменная: ответ или не ответ на вопрос (бинарная)

Значения ячеек - текущие рейтинги игроков

Строка датасета - вектор размера [количество игроков в данном турнире]

Турниры перебирались по очереди. При этом каждый раз строился новый обучающий датасет   

Данная схема была опробована на первом турнире с подсчетом функциии правдоподобия. Результат приведен ниже. Функция правдободобия при таком подходе уменьшалась на первых двух итерациях и далее возрастала.

In [139]:
tournament.head()

,round,team,mask,rating
0,4772,"[6212, 18332, 18036, 22799, 15456, 26089]",111111111011111110111111111100010010,"[13507, 13185, 12801, 12801, 12757, 12416]"
1,4772,"[1585, 40840, 1584, 10998, 16206]",111111111011110100101111011001011010,"[13058, 9138, 9030, 8863, 8732]"
2,4772,"[23513, 18168, 21060, 35850, 31332, 10187]",111111111011110101101111001011110000,"[9584, 9437, 9358, 9306, 8571, 6539]"
3,4772,"[36742, 28939, 54289, 15381, 27375]",101111101111111110001101011001111010,"[8592, 8258, 7995, 7455, 7436]"
4,4772,"[28689, 17720, 30597, 12400, 26988, 69476]",111111101011111101000111001001111110,"[12069, 9712, 8717, 8678, 8329, 8180]"


In [297]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [438]:
def likelihood(df, players_weights, y):
    
    eps = 10e-6
    
    wx = df[['team', 'rating']].apply(lambda t: sum([players_weights[p] * r 
                                                    for p, r in zip(t['team'], t['rating'])]), axis=1)
    
    L = sum(y * np.log(sigmoid(wx) + eps) + (1 - y) * np.log(1 - sigmoid(wx) + eps))
    
    return L

In [439]:
num_iterations = 10
tournament_length = max(tournament['mask'].apply(lambda t: len(t)).values)

team_rating = tournament[['team', 'rating']]

for i in range(num_iterations):
    
    likelihoods = []

    for question in range(tournament_length):
        y = tournament['mask'].apply(lambda t: int(t[question])).values
        if 0 not in y or 1 not in y:
            continue
        if set(y) != {0, 1}:
            corrections = set(y).difference({0, 1})
            print(f'{set(y)}; found corrections {corrections}')
            for c in corrections:
                y[np.where(y == c)] = 0
            
        log_reg.fit(members_df, y)
        logistic_weights = log_reg.coef_
        player_weight = {player: weight for player, weight in zip(members_df.columns.values, logistic_weights[0,:])}
        
        current_likelihood = likelihood(team_rating, player_weight, y)
        
        updated_ratings = team_rating[['team', 'rating']].apply(lambda t: 
                                                                [(player_weight_dict[p] + 1) * r 
                                                                 for p, r in zip(t['team'], t['rating'])], 
                                                                axis=1)
        
        team_rating['rating'] = updated_ratings
        
        likelihoods.append(likelihood(team_rating, player_weight, y))
        
        for ind in tournament.index:
            members_df.loc[ind, np.array(team_rating['team'].loc[ind])] = updated_ratings[ind]
            
    print(f'likelihood: {np.mean(likelihoods)}')

likelihood: -0.06203790551946547
likelihood: 0.002197628745367998
likelihood: -0.0545104146734597
likelihood: -1.762948260692563
likelihood: -2.1902987380764802
likelihood: -1.5340723820754334
likelihood: -1.2201852082690612
likelihood: -0.4324577998578396
likelihood: -0.32175111763356656
likelihood: -0.24232548761264283


Вспомогательные функции для обработки серии вычслений:

In [3]:
# Функция для выравнивания последовательностей вопросов

def align_tournament_length(tour_id):
    tour = masks2019[masks2019['round']==tour_id]
    mask_lengths = set(list(tour['mask'].apply(lambda t: len(t)).values))
    
    if len(mask_lengths) > 1:
        min_length = min(mask_lengths)
        return tour['mask'].apply(lambda t: t[:min_length])
    else:
        return tour['mask']

In [5]:
# Функция для проверки правильности целевой переменной

def process_answer(s):
    if s not in ['0', '1']:
        return 0
    else:
        return int(s)

In [7]:
# Функция для инициализации датасета для обучения логистической регрессии

def rating_init(tournament_id, source):
    tournament = masks2019[masks2019['round']==tournament_id]
    tournament = tournament.drop(tournament[tournament['team'].apply(lambda t: len(t)==0)].index)
    round_members = np.unique(np.hstack(tournament['team'].values))

    members = pd.DataFrame(
        index=tournament.index, 
        columns=sorted(round_members), 
        data=np.zeros((tournament.shape[0], round_members.shape[0]))
    )
    
    for ind in members.index:
        team = np.array(tournament['team'].loc[ind])
        members.loc[ind, team] = np.array(source.loc[team])
        
    return members, tournament, round_members

In [29]:
log_reg = LogisticRegression()

num_iterations = 6

parameters_df = pd.DataFrame(index=players_results2019.index, columns=['init_rating'])
parameters_df['init_rating'] = players_results2019['StartRating']
parameters_df['updated_rating'] = 10 * parameters_df['init_rating'] / (parameters_df['init_rating'].max() - 
                                                                      parameters_df['init_rating'].min())
parameters_df['weights'] = 0

tournament_ids = np.unique(masks2019['round'].values)
questions_df = pd.DataFrame(columns=['tournament', 'number', 'score', 'ratio'])
metrics = pd.DataFrame(index=range(num_iterations), columns=['spearman', 'kendall'])

    
for i in tqdm_notebook(range(num_iterations)):
    
    for tourn_id in tqdm_notebook(tournament_ids):
        if tourn_id == tournament_ids[663]:
            continue
            
        ratings, tournament, round_members = rating_init(tourn_id, parameters_df['updated_rating'])
        if tournament.shape[0] < 5:
            continue
            
        if np.any(tournament['mask'].apply(lambda t: set(t) == {0} or set(t) == {1})):
            continue            
            
        tournament['mask'] = align_tournament_length(tourn_id)
        tournament_length = max(tournament['mask'].apply(lambda t: len(t)).values)

        for question in range(tournament_length):
            y = tournament['mask'].apply(lambda t: process_answer(t[question])).values
            if 0 not in y or 1 not in y:
                continue
                
            if set(y) == {0} or set(y) == {1}:
                continue
            
            log_reg.fit(ratings, y)
            lw = log_reg.coef_.squeeze()
            player_weight = {player: weight for player, weight in zip(ratings.columns.values, lw)}
            other_players = list(set(parameters_df.index.values) - set(ratings.columns.values))
            player_weight.update({p: 0 for p in other_players})
            parameters_df['weights'] += parameters_df.index.map(player_weight)

            if i == num_iterations - 1:
                question_stats = pd.DataFrame(index=[0], columns=['tournament', 'number', 'score', 'ratio'])
                question_stats['tournament'].iloc[0] = tourn_id
                question_stats['number'].iloc[0] = question
                question_stats['score'].iloc[0] = question_scores(player_weight)
                question_stats['ratio'].iloc[0] = y.sum() / y.shape[0]
                
                if questions_df.empty:
                    questions_df = question_stats.copy()
                else:
                    questions_df = questions_df.append(question_stats)
                    
        parameters_df['weights'] = parameters_df['weights'] / y.shape[0]
        parameters_df['updated_rating'] = parameters_df['updated_rating'] * (parameters_df['weights'] + 1)
        parameters_df['weights'] = 0
        
    predicted_rating = parameters_df['updated_rating']
    spearman = spearmanr(players_results2019['FinalRating'], predicted_rating).correlation
    kendall = kendalltau(players_results2019['FinalRating'], predicted_rating).correlation
    metrics.iloc[i]['spearman'] = spearman
    metrics.iloc[i]['kendall'] = kendall
    print(f'Iteration {i} >> Kendall correlation: {kendall}, Spearman correlation: {spearman}')


Iteration 0 >> Kendall correlation: 0.7980537773458328, Spearman correlation: 0.8734834393363649



Iteration 1 >> Kendall correlation: 0.7965013916646637, Spearman correlation: 0.8733002560404297



Iteration 2 >> Kendall correlation: 0.794897786874282, Spearman correlation: 0.8730082149604741



Iteration 3 >> Kendall correlation: 0.7934590699348099, Spearman correlation: 0.8726952552890871



Iteration 4 >> Kendall correlation: 0.7921831203441092, Spearman correlation: 0.8723891338610005



Iteration 5 >> Kendall correlation: 0.7910275019562041, Spearman correlation: 0.8720887360633514



In [30]:
metrics

,spearman,kendall
0,0.873483,0.798054
1,0.8733,0.796501
2,0.873008,0.794898
3,0.872695,0.793459
4,0.872389,0.792183
5,0.872089,0.791028


In [33]:
parameters_df

,init_rating,updated_rating,weights
15,5077,6.87652,0
16,4247,4.19075,0
23,791,0.55291,0
31,4340,10.5204,0
35,5742,24.8176,0
...,...,...,...
224404,0,0,0
224408,0,0,0
224482,0,0,0
224539,0,0,0


Заключение: к сожалению, метрики при сравнени с окончательным рейтингом не увеличивались, но при этом были существенно выше бейзлайна.

***5. Подтсчет рейтинга вопросов осуществлялся как разница сумм рейтингов ответивши и сумм рейтингов неответивших участников.***

In [84]:
tournament_ids = np.unique(masks2019['round'].values)
questions_rating = pd.DataFrame(columns=['tournament', 'number', 'score', 'ratio'])

for tourn_id in tqdm_notebook(tournament_ids):
    if tourn_id == tournament_ids[663]:
        continue

    ratings, tournament, round_members = rating_init(tourn_id, parameters_df['init_rating'])
    if tournament.shape[0] < 5:
        continue

    if np.any(tournament['mask'].apply(lambda t: set(t) == {0} or set(t) == {1})):
        continue            

    tournament['mask'] = align_tournament_length(tourn_id)
    tournament_length = max(tournament['mask'].apply(lambda t: len(t)).values)

    for question in range(tournament_length):
        y = tournament['mask'].apply(lambda t: process_answer(t[question])).values
        if 0 not in y or 1 not in y:
            continue

        if set(y) == {0} or set(y) == {1}:
            continue

        question_stats = pd.DataFrame(index=[0], columns=['tournament', 'number', 'score', 'ratio'])
        question_stats['tournament'].iloc[0] = tourn_id
        question_stats['number'].iloc[0] = question
        question_stats['score'].iloc[0] = ratings.iloc[y[y==0]].sum().sum() - ratings.iloc[y[y==1]].sum().sum()
        question_stats['ratio'].iloc[0] = y.sum() / y.shape[0]

        if questions_rating.empty:
            questions_rating = question_stats.copy()
        else:
            questions_rating = questions_rating.append(question_stats)

In [88]:
questions_rating.iloc[np.argsort(questions_rating['score'])[-10:]]

,tournament,number,score,ratio
0,5757,46,7.25856e+07,0.0181612
0,5757,5,7.28674e+07,0.015891
0,5757,6,7.31492e+07,0.0136209
0,6255,4,7.51208e+07,0.193629
0,5760,10,7.55527e+07,0.289583
0,6255,20,7.95175e+07,0.179194
0,5760,11,9.51406e+07,0.226562
0,5760,22,9.85401e+07,0.215625
0,5760,14,1.07929e+08,0.185417
0,5760,15,1.23794e+08,0.134375


1: В романе Га́рри Га́ррисона описана биологическая цивилизация, которая модифицирует животных для решения повседневных задач. Слизень, который легко переваривает определённый белок, используется в качестве НЕЁ. Назовите ЕЁ, не используя лишних слов.

2: Фонтан, работавший на открытии Всесоюзной сельскохозяйственной выставки, украшали ИКСЫ. Аналогом ИКСА был горшок богини Ла́кшми. Назовите ИКС двумя словами.

3: По меткому замечанию, ИКС ставит на человеке крест. ИКСЫ производит, например, Nikon [ни́кон]. Назовите ИКС двумя словами.

4: В конце 2017 года пятилетний Та́йлон Пи́тман позвонил в службу спасения и сообщил, что необходимо остановить ПАЖА, пока не случилось что-то ужасное. Некоторые по ошибке называют ПАЖА ПАРИЖЕМ. Какие слова мы заменили словами «ПАЖ» и «ПАРИЖ»?

5: Турнир ОВСЧ-2019 не найден в базе

6: В семнадцатом веке точный состав воздуха ещё не был известен. Джон Ме́йоу считал, что при смешивании вдыхаемого воздуха и крови образуется аналог ИКСА, который вызывает сокращение мышц. Премьера сериала «ИКС» на канале BBC [би би си] состоялась осенью. Назовите ИКС словом с одинаковыми гласными.

7: Турнир ОВСЧ-2019 не найден в базе

8: Внимание, черный ящик. В своем объяснении Шэнь Ко использовал метафору уключины, когда при гребке веслом лопасть перемещается в противоположную сторону. Под веслом Шэнь Ко понимал ИКС. Какое слово заменено ИКСОМ?

9: Лидия Скрябина пишет, что ссылка заключенных на каторгу была обычным делом для Англии, так как Англия не располагала АЛЬФАМИ. В 2008 году отбывший срок человек тоже сослался на АЛЬФЫ. Что такое АЛЬФА?

10: Аргентинская ху́нта в 70-х годах устраняла тех, кто выступал против её власти, а их маленьких детей отдавала на усыновление военным. Через несколько лет после падения хунты организация, объединявшая волко́в, обратилась за помощью к учёным из США. Кого в этом вопросе мы заменили волком?